# Keyword Filters and Searches

In [11]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import utils
import weaviate.classes as wvc
from weaviate.classes.query import Filter

In [17]:
# KEWWORD FILTERING

client = utils.connect_to_demo_db()  # Connect to the demo database

movies = client.collections.get("Movie")

filter = Filter.by_property("description").like("thriller")  # Define the filter - look for the word "love" in the description
response = movies.query.fetch_objects(
    filters=filter,
    limit=3
)

for o in response.objects:
    print(o.properties["title"])                # Show which titles were found
    print(o.properties["description"], "\n")    # Show the description


#client.close()

Midnight Masquerade
A thriller set in 18th century Venice where masked balls hide more than just faces. 

Quantum Nexus
A mind-bending thriller about a scientist who discovers a way to enter alternate realities, but soon realizes the consequences are dire. 



In [19]:
# KEYWORD SEARCH

movies = client.collections.get("Movie")

response = movies.query.bm25(  # Keyword search for the word "love"
    query="thriller",
    limit=3,
    return_metadata=wvc.query.MetadataQuery(score=True),
)

for o in response.objects:
    print(o.properties["title"])        # Show which titles were found
    print(o.properties["description"])  # Show the description
    print(f"{o.metadata.score:.3f}\n")  # What was the score?


#client.close()

Midnight Masquerade
A thriller set in 18th century Venice where masked balls hide more than just faces.
1.212

Quantum Nexus
A mind-bending thriller about a scientist who discovers a way to enter alternate realities, but soon realizes the consequences are dire.
1.011



In [ ]:
# VECTOR SEARCH

for query in ["love", "amorous", "adventure movie set in outer galaxy"]:  # Loop through multiple query terms
    response = movies.query.near_text(  # Vector search
        query=query,
        limit=2,
        return_metadata=wvc.query.MetadataQuery(distance=True),
    )

    print(f"===== Search results for '{query}'. =====")  # Print the query term
    for o in response.objects:
        print(o.properties["title"])            # Show which titles were found
        print(o.properties["description"])      # Show the description
        print(f"{o.metadata.distance:.3f}\n")   # What was the distance?

In [ ]:
# SEARCH WITH FILTERS

# Define the filter - look for the word "love" in the description
filter = (
    wvc.query.Filter.by_property("year").greater_or_equal(1990)
    & wvc.query.Filter.by_property("description").like("space")
)

response = movies.query.near_text(  # Vector search
    query="science fiction",
    limit=2,
    filters=filter  # With the filter
)

for o in response.objects:
    movie_id = o.properties["movie_id"]
    movie_title = o.properties["title"]
    movie_year = o.properties["year"]

    print(f"ID: {movie_id}, {movie_title}, year: {movie_year}")     # Show which titles were found
    print(o.properties["description"][:50] + "...\n")                 # Show the description

In [ ]:
# HYBRID SEARCH

response = movies.query.hybrid(  # Hybrid search
    query="stellar",
    limit=3,
    # Fetch the score and explain_score
    return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
)

for o in response.objects:
    print(o.properties["title"])                        # Show which titles were found
    print(f"score: {o.metadata.score:.3f}")             # What was the score
    print(f"explain_score: {o.metadata.explain_score}") # Explain the score
    print()


alpha = 0  # Effectively a keyword search
response = movies.query.hybrid(
    query="stellar",
    limit=3,
    alpha=alpha,
    return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
)

print(f"===== Results with alpha: {alpha} =====")
for o in response.objects:
    print(o.properties["title"])  # Show which titles were found
    print(f"score: {o.metadata.score:.3f}")  # What was the distance?
    print(f"explain_score: {o.metadata.explain_score}\n")  # What was the distance?

In [ ]:
# RETRIVEL AUGMNETATED GENETRATED SEARCH

response = movies.generate.near_text(
    query="science fiction",
    limit=3,
    single_prompt="""
    Summarize the description:
    {description} for this movie {title}.
    """
)

for o in response.objects:
    print(o.properties["title"])    # Show which titles were found
    print(o.generated)              # RAG output
    print()


response = movies.generate.near_text(
    query="science fiction",
    limit=10,
    grouped_task="""
    Are there any common themes in these movies?
    Explain 2 in very short points,  and list the relevant movies:
    """
)

print(response.generated)  # Print the generated text

In [ ]:
# CHALLENGE

response = movies.query.fetch_objects(limit=1)

for o in response.objects:
    print(f"Movie: {o.properties['title']}\n")

# ========== TASKS ==========

# Task 1:
# Write a hybrid search query on "Review" collection.
# This collection has just the one property ("body").
#
# The query can be any text.
# e.g. “fun for the whole family”.
#
# Try this with two different alpha values, 0.1 and 0.9.
# Retrieve & display the top 3 results & scores.

reviews = client.collections.get("Review")

for alpha in [0.1, 0.9]:
    response = reviews.query.hybrid(
        query="fun for the whole family",
        alpha=alpha,
        limit=3,
        return_metadata=wvc.query.MetadataQuery(score=True),
    )

    print(f"Search results with alpha: {alpha}")
    for o in response.objects:
        print(f"Review body: {o.properties['body']}")
        print(f"Score: {o.metadata.score}\n")


# Task 2:
# A RAG query, on the “Movie” collection
# Using a vector search for: “action adventure”.
# Fetch the top 5 results, and
# Prompt the language model to generate a tagline for each movie,
# based on the movie’s title and description properties.

response = movies.generate.near_text(
    query="action adventure",
    limit=3,
    single_prompt="""
    Suggest a tagline for this film based on the title and description.
    Title: {title}
    Description: {description}
    """,
)

print(f"Generated results:")
for o in response.objects:
    print("---- Movie informaton ----")
    print(o.properties["title"])
    print(o.properties["description"])
    print("---- Generated tagline ----")
    print(o.generated)
    print()


client.close()